In [145]:
import pandas as pd
import numpy as np

In [146]:
df1 = pd.read_csv('2024_feb_unduplicated_orders.csv')
df1['title'] = df1['title'].str.lstrip()
df1['title'] = df1['title'].str.rstrip()
df1.shape

(59185, 22)

In [147]:
df1_notnull = df1[df1['customs_description'].notnull()]
df1_null = df1[df1['customs_description'].isna()]
df1_notnull.shape, df1_null.shape

((56557, 22), (2628, 22))

In [148]:
56557 + 2628

59185

In [149]:
temp_df = pd.read_csv('product_feed.csv')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&gt;', '>')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&amp;', '&')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&#039;', "'")
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&gt;', '>')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&amp;', '&')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&#039;', "'")
temp_df.rename(columns={'TITLE': 'title'}, inplace=True)
temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]
temp_df['title'] = temp_stock_df[0]
temp_df['title'] = temp_df['title'].str.lstrip()
temp_df['title'] = temp_df['title'].str.rstrip()
temp_df.drop_duplicates(subset=['title'], inplace=True)
temp_df = temp_df[['title', 'PRODUCT TYPE']]

<>:9: SyntaxWarning: invalid escape sequence '\('
<>:9: SyntaxWarning: invalid escape sequence '\('
C:\Users\retai\AppData\Local\Temp\ipykernel_12928\526015662.py:9: SyntaxWarning: invalid escape sequence '\('
  temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]


In [150]:
temp_df[temp_df['title'] == "Women's Collarless Leather Fashion Jacket Black: AWL-284"]

,title,PRODUCT TYPE
9091,Women's Collarless Leather Fashion Jacket Blac...,Ladies > Leather Jackets > Casual Styles


In [151]:
temp_df['title'].str.contains("Women's Collarless Leather Fashion Jacket Black: AWL-284").any()

True

In [152]:
df1_null = df1_null.merge(temp_df, on='title', how='left')
df1_null['customs_description'] = df1_null['PRODUCT TYPE']
df1_null.drop(['PRODUCT TYPE'], axis=1, inplace=True)
df1_null.shape

(2628, 22)

In [153]:
df1_null[df1_null['title'] == 'Mens Leather Hodded Jacket Cognac: Super Bomber']

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,F_Cat,S_Cat,T_Cat,Category,colour
2344,76027,Order Dispatched,2024-01-14 15:52:23,200.0,Mens Leather Hodded Jacket Cognac: Super Bomber,90495,super bomber,4948007,1,175.0,...,Mens > Leather Jackets > Blouson & Casual Styles,2024-01-14,15:52:23,14,1,NaN,NaN,NaN,NaN,Cognac


In [154]:
df1 = pd.concat([df1_notnull, df1_null], ignore_index=True)
df1.shape

(59185, 22)

In [155]:
df1[df1['title'] == "Women's Collarless Leather Fashion Jacket Black: AWL-284"]

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,F_Cat,S_Cat,T_Cat,Category,colour
57205,67906,Order Dispatched,2023-10-13 17:47:17,269.0,Women's Collarless Leather Fashion Jacket Blac...,80444,awl-284,4861307,1,269.0,...,Ladies > Leather Jackets > Casual Styles,2023-10-13,17:47:17,13,1,NaN,NaN,NaN,NaN,Black


In [156]:
df1['customs_description'].isnull().sum()

1308

In [157]:
df1.shape

(59185, 22)

In [158]:
df1.drop_duplicates(inplace=True)
df1.shape

(59185, 22)

In [159]:
df2 = df1['customs_description'].str.split(' > ', expand=True)
df1.drop(['Category', 'F_Cat', 'S_Cat', 'T_Cat'], axis=1, inplace=True)
df2.columns = ['F_Cat', 'S_Cat', 'T_Cat']
df1 = df1.join(df2)
df1['Category'] = df1['F_Cat'] + ' > ' + df1['S_Cat']

In [160]:
df1[df1['title'] == 'Mens Leather Hodded Jacket Cognac: Super Bomber']

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,colour,F_Cat,S_Cat,T_Cat,Category
58901,76027,Order Dispatched,2024-01-14 15:52:23,200.0,Mens Leather Hodded Jacket Cognac: Super Bomber,90495,super bomber,4948007,1,175.0,...,Mens > Leather Jackets > Blouson & Casual Styles,2024-01-14,15:52:23,14,1,Cognac,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets


In [161]:
df1[df1['title'] == "Women's Collarless Leather Fashion Jacket Black: AWL-284"]

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,colour,F_Cat,S_Cat,T_Cat,Category
57205,67906,Order Dispatched,2023-10-13 17:47:17,269.0,Women's Collarless Leather Fashion Jacket Blac...,80444,awl-284,4861307,1,269.0,...,Ladies > Leather Jackets > Casual Styles,2023-10-13,17:47:17,13,1,Black,Ladies,Leather Jackets,Casual Styles,Ladies > Leather Jackets


In [162]:
df1.to_csv('2024_feb_unduplicated_orders fixed_categories.csv')

In [163]:
df1[df1['title'] == 'Mens Leather Hodded Jacket Black: Super Bomber']

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,colour,F_Cat,S_Cat,T_Cat,Category
55530,77921,Awaiting Dispatch,2024-02-29 21:51:05,199.000,Mens Leather Hodded Jacket Black: Super Bomber,92816,super bomber,4946506,1,199.000,...,Mens > Leather Jackets > Bomber Styles,2024-02-29,21:51:05,29,1,Black,Mens,Leather Jackets,Bomber Styles,Mens > Leather Jackets
55692,77744,Order Dispatched,2024-02-24 11:46:12,189.050,Mens Leather Hodded Jacket Black: Super Bomber,92619,super bomber,4946505,1,199.000,...,Mens > Leather Jackets > Bomber Styles,2024-02-24,11:46:12,24,1,Black,Mens,Leather Jackets,Bomber Styles,Mens > Leather Jackets
55744,77687,Order Dispatched,2024-02-22 15:32:05,189.000,Mens Leather Hodded Jacket Black: Super Bomber,92548,super bomber,4946504,1,199.000,...,Mens > Leather Jackets > Bomber Styles,2024-02-22,15:32:05,22,1,Black,Mens,Leather Jackets,Bomber Styles,Mens > Leather Jackets
55779,77655,Order Dispatched,2024-02-21 20:29:09,190.825,Mens Leather Hodded Jacket Black: Super Bomber,92505,super bomber,4946506,1,165.830,...,Mens > Leather Jackets > Bomber Styles,2024-02-21,20:29:09,21,1,Black,Mens,Leather Jackets,Bomber Styles,Mens > Leather Jackets
57379,70654,Order Dispatched,2023-11-23 23:59:52,269.000,Mens Leather Hodded Jacket Black: Super Bomber,83922,super bomber,4946510,1,269.000,...,Mens > Leather Jackets > Blouson & Casual Styles,2023-11-23,23:59:52,23,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
57394,70546,Order Dispatched,2023-11-23 02:54:27,298.333,Mens Leather Hodded Jacket Black: Super Bomber,83781,super bomber,4946504,1,165.833,...,Mens > Leather Jackets > Blouson & Casual Styles,2023-11-23,02:54:27,23,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
57890,72860,Order Dispatched,2023-12-09 14:44:40,189.050,Mens Leather Hodded Jacket Black: Super Bomber,86661,super bomber,4946505,1,199.000,...,Mens > Leather Jackets > Blouson & Casual Styles,2023-12-09,14:44:40,9,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
57972,72244,Order Dispatched,2023-12-05 16:12:08,199.000,Mens Leather Hodded Jacket Black: Super Bomber,85886,super bomber,4946504,1,199.000,...,Mens > Leather Jackets > Blouson & Casual Styles,2023-12-05,16:12:08,5,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
58041,71636,Order Dispatched,2023-12-01 01:05:03,269.000,Mens Leather Hodded Jacket Black: Super Bomber,85149,super bomber,4946510,1,269.000,...,Mens > Leather Jackets > Blouson & Casual Styles,2023-12-01,01:05:03,1,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
59026,76745,Order Incomplete,2024-01-28 12:48:16,190.833,Mens Leather Hodded Jacket Black: Super Bomber,91388,super bomber,4946503,1,165.833,...,Mens > Leather Jackets > Blouson & Casual Styles,2024-01-28,12:48:16,28,1,Black,Mens,Leather Jackets,Blouson & Casual Styles,Mens > Leather Jackets
